In [1]:
import pandas as pd
from neo4j import GraphDatabase
import pymongo

### Conexiones

In [2]:
#Neo4j
neo_connection = GraphDatabase.driver(
    uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
session = neo_connection.session()    

In [3]:
#mongo
client=pymongo.MongoClient('mongodb://127.0.0.1:27017/')
mydb = client['Practica4']
table = mydb["tabla_peliculas"]

### Consultas

#### Consulta 1

In [39]:
#actores que trabajaron juntos
ac = ['Raymond Hatton', 'Roy Rogers']

#Dados dos actores, obtener peliculas en las que trabajaron juntos.
c0 = f"\
match (a:actor {{nombre: '{ac[0]}'}}) \
match (b:actor {{nombre: '{ac[1]}'}}) \
match (t:titulo) \
match (a) -- (t) -- (b) \
return (t)\
"
c0

"match (a:actor {nombre: 'Raymond Hatton'}) match (b:actor {nombre: 'Roy Rogers'}) match (t:titulo) match (a) -- (t) -- (b) return (t)"

In [42]:
def consulta1(ac):
    c0 = f"\
match (a:actor {{nombre: '{ac[0]}'}}) \
match (b:actor {{nombre: '{ac[1]}'}}) \
match (t:titulo) \
match (a) -- (t) -- (b) \
return (t)\
"

    a = session.run(c0).data()

    consulta = []
    for i in a:
        for j in table.find({'id_pelicula':i["t"]["id_titulo"]}):
            consulta.append(j)

    df = pd.DataFrame.from_dict(consulta)
    df = df[["título","descripción","calificación_imdb"]]

    return df

In [43]:
result = consulta1(ac)
result

,título,descripción,calificación_imdb
0,"Come On, Rangers",A Texas Ranger (Roy Rogers) and his pals come ...,6.2
1,Wall Street Cowboy,"When his ranch falls on hard times, Cowboy Roy...",6.0
2,Frontier Pony Express,"In the midst of the Civil War, Lassiter has a ...",6.2
3,Rough Riders' Round-up,Roy Rogers is a cowboy who joins the Border Pa...,5.4


#### Consulta 2

In [44]:
director = 'Wallace Fox'

#Dado un director, encuentra actores que fueron dirigidos por él.
c1 = f"\
match (a:actor) \
match (d:director {{nombre: '{director}'}}) \
match (t:titulo) \
match (d) -- (t) -- (a) \
return a;\
"
c1

"match (a:actor) match (d:director {nombre: 'Wallace Fox'}) match (t:titulo) match (d) -- (t) -- (a) return a;"

In [56]:
#Prueba función de consulta2

def consulta2(director,cantidad):
    c1 = f"\
match (a:actor) \
match (d:director {{nombre: '{director}'}}) \
match (t:titulo) \
match (d) -- (t) -- (a) \
return a, t;\
"

    a = session.run(c1).data()
    consulta = []
    c = []
    d = []
    for v in a:
        consulta.append(v["a"]["nombre"])
        c.append(v["t"]["id_titulo"])

    for j,v in enumerate(c):
        for k in table.find({'id_pelicula':v}):
            #print(k)
            d.append(k)

    d1 = pd.DataFrame.from_dict(consulta)
    d2 = pd.DataFrame.from_dict(d)
    d2 = d2[["título","descripción","calificación_imdb"]]
    resultado = pd.concat([d1,d2],axis=1)

    return resultado[:cantidad]

In [57]:
consulta2(director, 8)

,0,título,descripción,calificación_imdb
0,Robert 'Buzz' Henry,The Great Mike,Story of a boy and his horse. Mike is the hors...,5.3
1,Stuart Erwin,The Great Mike,Story of a boy and his horse. Mike is the hors...,5.3
2,Eddie Kane,Career Girl,"Joan Terry, from Kansas City, comes to New Yor...",5.5
3,Bess Flowers,Career Girl,"Joan Terry, from Kansas City, comes to New Yor...",5.5
4,Marcy McGuire,Career Girl,"Joan Terry, from Kansas City, comes to New Yor...",5.5
5,Renee Helms,Career Girl,"Joan Terry, from Kansas City, comes to New Yor...",5.5
6,Charles Williams,Career Girl,"Joan Terry, from Kansas City, comes to New Yor...",5.5
7,Charles Judels,Career Girl,"Joan Terry, from Kansas City, comes to New Yor...",5.5


#### Consulta 3

In [58]:
#Dadas dos peliculas, encontrar a quienes actuaron en ambas:

pc = ["Come On, Rangers", "Wall Street Cowboy"]

c2 = f"\
match (a:actor) \
match (p:movie {{nombre:'{pc[0]}'}}) \
match (k:movie {{nombre:'{pc[1]}'}}) \
match (p) -- (a) -- (k) \
return (a) \
"
c2

"match (a:actor) match (p:movie {nombre:'Come On, Rangers'}) match (k:movie {nombre:'Wall Street Cowboy'}) match (p) -- (a) -- (k) return (a) "

In [62]:
def consulta3(pc,cantidad):
    c2 = f"\
match (a:actor) \
match (p:movie {{nombre:'{pc[0]}'}}) \
match (k:movie {{nombre:'{pc[1]}'}}) \
match (p) -- (a) -- (k) \
return (a) \
"
    a = session.run(c2).data()
    consulta = []
    for v in a:
        consulta.append(v["a"]["nombre"])
    return consulta[:cantidad]

In [65]:
consulta3(pc, 10)

['Raymond Hatton', 'Roy Rogers']